In [11]:
import numpy as np
import timeit

from sklearn.datasets import make_spd_matrix

In [2]:
class Gershgorin:
    def estimate_eigenvalues(self, A: np.ndarray):
        first_circles = []
        second_circles = []
        
        union_circles = []
        
        for row in range(A.shape[0]):
            current_radius = 0
            for column in range(A.shape[1]):
                if row == column:
                    continue
                
                current_radius += abs(A[row][column])
                
            first_circles.append([A[row][row] - current_radius, A[row][row] + current_radius])
        
        for column in range(A.shape[1]):
            current_radius = 0
            for row in range(A.shape[0]):
                if row == column:
                    continue
                
                current_radius += abs(A[row][column])
                
            second_circles.append([A[row][row] - current_radius, A[row][row] + current_radius])
            
        for first_circle, second_circle in zip(first_circles, second_circles):
            union_circles.append([max(first_circle[0], second_circle[0]), min(first_circle[1], second_circle[1])])
        
        lower_bound = min(map(lambda x: x[0], union_circles))
        upper_bound = max(map(lambda x: x[1], union_circles))
        
        return max(0, lower_bound), upper_bound

In [19]:
class Richardson:
    def solve(self, A: np.ndarray, b: np.ndarray, tau: float, tolerance: float = 1e-3, max_steps: int = None):
        assert A.shape[0] == A.shape[1]
        assert A.shape[0] == b.shape[0]
        assert b.shape[1] == 1
        
        start = timeit.default_timer()
        
        error = tolerance + 1.0
        steps = 0
        current_x = np.ones_like(b)
        S = (np.identity(A.shape[0]) - tau * A)
        
        while error >= tolerance and (max_steps is None or steps < max_steps):
            current_x = S @ current_x + tau * b
            
            error = np.linalg.norm(A @ current_x - b)
            steps += 1
            
        if error < tolerance:
            print(f"Tolerance level reached with error: {error} and {steps} steps")
        if steps == max_steps:
            print(f"Max steps {max_steps} reached")
            
        stop = timeit.default_timer()
        
        print('Time elapsed:', stop - start)
            
        return current_x

In [23]:
MATRIX_SIZE = 150
TOLERANCE = 1e-4

A = make_spd_matrix(MATRIX_SIZE)
b = np.random.randn(MATRIX_SIZE, 1)

start = timeit.default_timer()
numpy_solution = np.linalg.solve(A, b)
stop = timeit.default_timer()

print('Numpy solution time elapsed:', stop - start)

gershgorin = Gershgorin()
print("Eigenvalues estimate by Gershgorin circles:", gershgorin.estimate_eigenvalues(A))
print("Min and max eigenvalues from Numpy:", np.linalg.eigvals(A).min(), np.linalg.eigvals(A).max())

Numpy solution time elapsed: 0.11212330000000748
Eigenvalues estimate by Gershgorin circles: (0, 338.1072583350118)
Min and max eigenvalues from Numpy: 0.004900046915039872 150.4861771586605


## Arbitrary tau

In [24]:
tau = 1 / np.linalg.eigvals(A).max()
print('Computing with tau:', tau)

my_solution = Richardson().solve(A, b, tau)

print('Norm of difference with Numpy solution:', np.linalg.norm(my_solution - numpy_solution))

Computing with tau: 0.006645128601716559
Tolerance level reached with error: 0.000999979051535903 and 167847 steps
Time elapsed: 131.03480109999998
Norm of difference with Numpy solution: 0.2040753602155222


## Estimate optimal tau

In [25]:
min_eigenvalue, max_eigenvalue = gershgorin.estimate_eigenvalues(A)
tau = 2 / (min_eigenvalue + max_eigenvalue)
print('Computing with tau:', tau)

my_solution = Richardson().solve(A, b, tau)

print('Norm of difference with Numpy solution:', np.linalg.norm(my_solution - numpy_solution))

Computing with tau: 0.005915282652756039
Tolerance level reached with error: 0.0009999738760021518 and 188557 steps
Time elapsed: 145.06775129999983
Norm of difference with Numpy solution: 0.20407430400019877


## Exact optimal tau

In [26]:
min_eigenvalue = np.linalg.eigvals(A).min()
max_eigenvalue = np.linalg.eigvals(A).max()

tau = 2 / (min_eigenvalue + max_eigenvalue)
print('Computing with tau:', tau)

my_solution = Richardson().solve(A, b, tau)

print('Norm of difference with Numpy solution:', np.linalg.norm(my_solution - numpy_solution))

Computing with tau: 0.013289824467585792
Tolerance level reached with error: 0.0009999563786070068 and 183483 steps
Time elapsed: 144.63476960000003
Norm of difference with Numpy solution: 0.00031198610654800717
